In [1]:
import numpy as np
import random

In [2]:
def eig(A_original, max_iter = 100, atol=1e-5, verbose=False):
    A = A_original.copy().astype(float)
    current_A = A.copy()
    n = A.shape[0]
    eigenvalues = []
    eigenvectors = np.zeros_like(A)
    for i in range(n):
        v = np.random.rand(n)
        v = v/np.linalg.norm(v)
        mu = np.random.normal(0, 10)
        current_A = A - sum(eigenvalues[j] * np.outer(eigenvectors[:, j], eigenvectors[:, j].T) for j in range(i))
        iterations = 0
        while np.linalg.norm(current_A @ v - mu * v) > atol and iterations < max_iter:
            try:
                v = np.linalg.solve(current_A - mu * np.identity(n), v)
            except np.linalg.LinAlgError:
                if verbose:
                    print(f"Invalid eigenvalue {i+1}")
                mu = np.random.normal(0, 10)
                continue
            v = v/np.linalg.norm(v)
            mu = v.T@current_A@v
            if abs(mu)<atol:
                mu = np.random.normal(0, 10)
            iterations += 1
        if iterations >= max_iter and verbose:
            print(f"Code failed to converge for eigenvalue {i+1}.")
        eigenvectors[:,i] = v
        eigenvalues.append(mu)
    return (np.array(eigenvalues), eigenvectors)

In [3]:
A = np.array([[1, 2, 3],
              [2, 5, 4],
              [3, 4, 10]])
code_eigenvalues, code_eigenvectors = eig(A, max_iter = 100)

print('Wartości własne kodu:\n', code_eigenvalues, '\nWektory własne kodu:\n', code_eigenvectors, '\nlinalg:\n', np.linalg.eig(A))

Wartości własne kodu:
 [13.27009751 -3.74085042  3.74084509] 
Wektory własne kodu:
 [[-0.28122477  0.27336177  0.2735275 ]
 [-0.47210708  0.47391133  0.47387361]
 [-0.83548042  0.83706714  0.83703435]] 
linalg:
 EigResult(eigenvalues=array([13.27009751, -0.08044285,  2.81034534]), eigenvectors=array([[ 0.28122477,  0.95520076, -0.09221791],
       [ 0.47210705, -0.22137446, -0.85329261],
       [ 0.83548044, -0.1964303 ,  0.51321299]]))


In [4]:
def random_matrices(columns, rows, quantity=1):
    result = []
    for i in range(quantity):
        result.append(np.random.rand(rows, columns))
    return np.array(result)

In [5]:
def validate(function, n=100, dim_max= 10, max_iter = 10000, verbose=False, atol=1e-2):
    success_count = 0
    for dim in range(2, dim_max+1):
        for A in random_matrices(dim, dim, n):
            array = A@ A.T
            result = np.sort(function(array, max_iter, verbose)[0])
            correct_result = np.sort(np.linalg.eig(array)[0])
            pairwise_distances = np.isclose(result[:, None], correct_result, atol=1e-2)
            if np.all(np.isclose(result, correct_result, atol=atol)):
                success_count+=1
            elif verbose:
                    print(f"array:\n{array}\nResult:\n{result}\nCorrect result:\n{np.linalg.eig(array)[0]}")
    return success_count/(n*dim_max)*100

In [6]:
print(f"Skuteczność kodu wynosi {validate(eig, n=100, dim_max= 5, max_iter = 100, verbose=False, atol=1e-2)}%")

Skuteczność kodu wynosi 11.600000000000001%


In [10]:
#Testowanie teorii
A = np.array([[1, 2, 3],
              [2, 5, 4],
              [3, 4, 10]])
x = np.linalg.eig(A)
print(x[0])
x2 = np.linalg.eig(A-x[0][0]*np.outer(x[1][:, 0], x[1][:, 0].T))
print(x2[0])
x3 = np.linalg.eig(A-x2[0][0]*np.outer(x2[1][:, 0], x2[1][:, 0].T))
print(x3[0])

[13.27009751 -0.08044285  2.81034534]
[ 2.81034534e+00 -8.04428504e-02 -2.51787677e-15]
[ 1.32700975e+01 -8.04428504e-02 -2.11492520e-15]
